##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ajuste de hiperparámetros con el panel HParams

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tensorboard/hyperparameter_tuning_with_hparams.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tensorboard/hyperparameter_tuning_with_hparams.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tensorboard/hyperparameter_tuning_with_hparams.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

Cuando se construyen modelos de aprendizaje automático, es necesario elegir varios [hiperparámetros](https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)), como la tasa de abandono en una capa o la tasa de aprendizaje. Estas decisiones influyen en las métricas del modelo, como puede ser la precisión. Por lo tanto, un paso importante en el flujo de trabajo del aprendizaje automático es identificar los mejores hiperparámetros para su problema, lo que a menudo implica experimentación. Este proceso se conoce como "optimización de hiperparámetros" o "ajuste de hiperparámetros".

El panel HParams de TensorBoard proporciona varias herramientas para ayudar en este proceso de identificación del mejor experimento o de los conjuntos de hiperparámetros más prometedores.

Este tutorial se centrará en los siguientes pasos:

1. Preparación del experimento y resumen de HParams
2. Adaptar las ejecuciones de TensorFlow para registrar hiperparámetros y métricas
3. Inicie las ejecuciones y regístrelas todas bajo un mismo directorio padre
4. Visualice los resultados en el panel HParams de TensorBoard

Nota: Las API de resumen de HParams y la interfaz de usuario del panel de control se encuentran en una fase preliminar y cambiarán con el tiempo.

Comience instalando TF 2.0 y cargando la extensión del bloc de notas de TensorBoard:

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

Importe TensorFlow y el complemento TensorBoard HParams:

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

Descargue el conjunto de datos [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) y escálelo:

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4423680/4422102 [==============================] - 0s 0us/step


## 1. Preparación del experimento y resumen del experimento HParams

Experimente con tres hiperparámetros en el modelo:

1. Número de unidades en la primera capa densa
2. Tasa de abandono en la capa de abandono
3. Optimizador

Enumere los valores a probar y registre una configuración del experimento en TensorBoard. Este paso es opcional: puede proporcionar información de dominio para permitir un filtro más preciso de los hiperparámetros en la interfaz de usuario, y puede especificar qué métricas deben mostrarse.

In [ ]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

Si decide omitir este paso, puede utilizar una cadena literal allí donde de otro modo utilizaría un valor `HParam`: por ejemplo, `hparams['dropout']` en vez de `hparams[HP_DROPOUT]`.

## 2. Adaptar las ejecuciones de TensorFlow para registrar hiperparámetros y métricas

El modelo será bastante simple: dos capas densas con una capa de abandono entre ellas. El código de entrenamiento le resultará familiar, aunque los hiperparámetros ya no están codificados. En cambio, los hiperparámetros se proporcionan en un diccionario `hparams` y se utilizan en toda la función de entrenamiento:

In [ ]:
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(x_train, y_train, epochs=1) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

Para cada ejecución, registre un resumen hparams con los hiperparámetros y la precisión final:

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

Cuando entrene modelos de Keras, puede utilizar retrollamadas en vez de escribirlas directamente:

```python
model.fit(
    ...,
    callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),  # log metrics
        hp.KerasCallback(logdir, hparams),  # log hparams
    ],
)
```

## 3. Inicie las ejecuciones y regístrelas todas bajo un mismo directorio padre

Ahora puede probar varios experimentos, entrenando cada uno con un conjunto diferente de hiperparámetros.

Para simplificar, utilice una búsqueda cuadriculada: pruebe todas las combinaciones de los parámetros discretos y sólo los límites inferior y superior del parámetro basado en valores reales. Para escenarios más complejos, puede ser más eficaz elegir cada valor del hiperparámetro al azar (esto se denomina búsqueda aleatoria). Se pueden utilizar métodos más avanzados.

Realice algunos experimentos, que le tomarán unos minutos:

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
10000/10000 [==============================] - 0s 35us/sample - loss: 0.4806 - accuracy: 0.8321
--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
10000/10000 [==============================] - 0s 36us/sample - loss: 0.6519 - accuracy: 0.7770
--- Starting trial: run-2
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'adam'}
10000/10000 [==============================] - 0s 36us/sample - loss: 0.5309 - accuracy: 0.8154
--- Starting trial: run-3
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'sgd'}
10000/10000 [==============================] - 0s 36us/sample - loss: 0.7007 - accuracy: 0.7683
--- Starting trial: run-4
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'adam'}
10000/10000 [==============================] - 0s 38us/sample - loss: 0.4645 - accuracy: 0.8395
--- Starting trial: run-5
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'sgd'}
10000/10000 [========================

## 4. Visualice los resultados en el complemento HParams de TensorBoard

Ahora puede abrir el panel de HParams. Inicie TensorBoard y haga clic en "HParams" en la parte superior.

In [ ]:
%tensorboard --logdir logs/hparam_tuning

<!-- <img class="tfo-display-only-on-site" src="images/hparams_table.png?raw=1"/> -->

El panel izquierdo del panel de control ofrece funciones de filtro que están activas en todas las vistas del panel de control de HParams:

- Filtre qué hiperparámetros/ métricas se mostrarán en el panel de control
- Filtre qué valores de hiperparámetros/métricas se mostrarán en el panel de control
- Filtro sobre el estado de ejecución (en marcha, éxito, ...)
- Ordenar por hiperparámetro/métrica en la vista en forma de tabla
- Número de grupos de sesiones a mostrar (útil para el rendimiento cuando hay muchos experimentos)


El panel de control de HParams tiene tres vistas diferentes, con información útil variada:

- La **Vista de la tabla** enumera las ejecuciones, sus hiperparámetros y sus métricas.
- La **Vista de coordenadas paralelas** muestra cada pasada como una línea que atraviesa un eje para cada hiperparámetro y métrica. Haga clic y arrastre el ratón sobre cualquier eje para marcar una región que resaltará sólo las ejecuciones que pasen por ella. Esto puede ser útil para identificar qué grupos de hiperparámetros son los más importantes. Los propios ejes pueden reordenarse arrastrándolos.
- La vista **Diagrama de dispersión** muestra gráficos que comparan cada hiperparámetro/métrica con cada métrica. Esto puede ayudar a identificar correlaciones. Haga clic y arrastre para seleccionar una región en un gráfico específico y resaltar esas sesiones en los demás gráficos.

Se puede hacer clic en una fila de la tabla, una línea de coordenadas paralelas y un mercado de gráficos de dispersión para ver un gráfico de las métricas en función de los pasos de entrenamiento para esa sesión (aunque en este tutorial sólo se utilice un paso para cada corrida).

Para explorar más a fondo las capacidades del panel de control de HParams, descargue un conjunto de registros pregenerados con más experimentos:

In [ ]:
%%bash
wget -q 'https://storage.googleapis.com/download.tensorflow.org/tensorboard/hparams_demo_logs.zip'
unzip -q hparams_demo_logs.zip -d logs/hparam_demo

Visualice estos registros en TensorBoard:

In [ ]:
%tensorboard --logdir logs/hparam_demo

<!-- <img class="tfo-display-only-on-site" src="images/hparams_parallel_coordinates.png?raw=1"/> -->

Puede probar las distintas vistas en el panel de control de HParams.

Por ejemplo, si va a la vista de coordenadas paralelas y hace clic y arrastra en el eje de precisión, puede seleccionar las ejecuciones con mayor precisión. Como estas ejecuciones pasan por 'adam' en el eje del optimizador, puede concluir que 'adam' funcionó mejor que 'sgd' en estos experimentos.